# Project: Simulation of a 2D gas

Clyde Fare and João Pedro Malhado, Imperial College London (contact: [python@imperial.ac.uk](mailto:python@imperial.ac.uk))

Notebook is licensed under a [Creative Commons Attribution 4.0 (CC-by) license](http://creativecommons.org/licenses/by/4.0/) except for the figures referenced.

A computational chemist friend of mine once defined simulations in the following terms:

<blockquote>Experimental work resembles the actions of a child that receives a big wraped present, and without opening it, by shaking it in his hands and hearing the sound it makes, has to infer what is inside the box and how it works. Simulation on the other hand, is like starting with an empty box, fill it with pieces that we know well, close and wrap the box, and shake it to hear the sound it makes. By comparing the two sounds, we may be able to understand nature a little better.
(Philipp Marquetand, sometime around 2009)</blockquote>

This project is about building a 2d box, filling it with disks, and hearing what sound it makes.

## The model

Similar to what Nick Brooks has done for the [Chemical Equilibria course](http://www.imperial.ac.uk/chemistry-teaching/chemicalequilibria/).

In this workshop we will work on a project to construct a simulation of a gas in a 2d box. You may have seen something similar on your lectures though here our simulation will be a bit simpler.

We will be simulating an ideal gas so particles will collide with each other and the walls of the box in a perfectly elastic manner. The aim will be to simulate N particles, in a box of size L &times; L, given some initial conditions, namely the positions and momementa of the particles involved.

Once we have our simulation up and running we'll check the distribution of speeds and see if it corresponds to the distribution predicted by the 2d version of the Maxwell-Boltzmann distribution.

Rather than working directly in the notebook we will build our classes inside a file: gas_2d.py using an editor we'll then visualise and test the code inside the notebook. To do that we'll need to execute a few helper utilities. Which will mean as soon as we update the code in our file it will be availbe within the notebook. Please execute these utilities in the cell below:

In [1]:
%load_ext autoreload
%autoreload 2

We've also provided some helper functions to visualise the simulation that you will construct and a slightly expanded Vector class that we will make use of.

In [3]:
import numpy as np
from oo_functions import animate_trajectory, display_particle, display_vecs, display_trajectory, Vector
from gas_2d import Particle, Simulation

Open gas_2d.py by using the Python editor Spyder (you can find it in your list of programs). Inside gas_2d.py you will find three class definitions, Vector, Particle and Simulation. 

Vector is already filled out and is much the same as the Vector class we developed in the last workshop with a few additional methods that means we can add, subtract, multiply and divide our vectors using the usual +-/\* operators. Particle and Simulation are just the outlines of the class. You will need to replace *pass* with code as you as you progress through the project.

## Building a Particle Class

The first class we need to fill out is the Particle class which will unsurprisingly represents *drum roll* particles. Once we can create particle objects we can start building our Simulation class which will represent the total simulation and will include multiple Particle objects.

For our purposes the properties that define a single particle will be position momentum, mass and radius. In the skeleton of the class below replace *pass* with code so that the particle class has position/momentum/radius attributes. We will want a method *velocity* which will return the velocity of the particle (recall that our expanded Vector class allows us to divide vectors by scalars). We will also want a method *copy* which will return an identical copy of the Particle shown below is a skeleton of the class we want to construct. Edit gas.py by filling out the particle class. (Once saved it should automatically be available to the notebook.

    class Particle():
        def __init__(self, position, momentum, radius, mass):
            pass

        def velocity(self):
            pass
            
        def copy(self):
            pass

We've created some helper functions for this workshop that will help with visualisation. One function is called display_particle. In the cell below use your particle class to create a particle object and then use the display_particle function on it to display it. 

In order to initialise your particle you'll need to create two vectors one for position and one for momentum and provide your particle with a radius and a mass.

A plot showing your particle should have popped up. Bask in the glory of the particle you have created.

## Making our particle move

Ok now we can create individual particles but they aren't doing too much, let's try generate a trajectory. As our particle is a free particle experiencing no forces the momentum will not change with time. The position however will change. In a small amount of time dt the position will change via:

$$\vec{r}(t+\Delta t)=\vec{r}(t)+\vec{v}(t)\times \Delta t$$

Where $\vec{r}(t)$ is the position at time $t$, $\vec{v}(t)$ is velocity and $\Delta t$ is the time step.

To generate a trajectory we are going to run our simulation for a certain number of time steps, during each step we will use the velocity to update the position and then we'll record the position in a list.

Fill out the cell so that during each iteration the particle's position is updated, you'll also want to append a copy of the particle to the trajectory list so that we have a record of the particle at every time step.

In [2]:
no_steps = 1000
dt = 0.01

trajectory = []
for step in range(no_steps):
    pass

Use the display_trajectory function on the positions list to view the motion of your particle

In [5]:
display_trajectory(trajectory)

## Building a Simulation class

Great! We're going to take a step sideways now and introduce our main simulation class. The code you wrote above calculated the position of a single particle for multiple time steps.

This time we're going to have many particles but we're only going to calculate their position for a single time step. Below is a skeleton for our simulation class, it will be initialised with a list of particles, a number box_length and another number dt. It includes a method *step* which is where the action takes place. The step method should loop over every particle and update the particle's position using it's momementum and the small amount time dt.

    class Simulation():
        def __init__(self, particles, box_length, dt):
            pass

        def step(self):
            pass

Test your simulation class using the code below (we're printing out the y coordinate of the first particle at the start of the simulation and after we have updated the positions). Do you understand what's being printed out?

In [7]:
particles = [Particle(Vector(0.5,0),Vector(0,1),1,1),
             Particle(Vector(0.5,1),Vector(0,-1),1,1),
             Particle(Vector(0,0), Vector(0.5,0.5),1,1)]

s=Simulation(particles, 100,1)

print(s.particles[0].position.y)
s.step()
print(s.particles[0].position.y)
s.step()
print(s.particles[0].position.y)

0
1.0
2.0


## Recording the state of the simulation

We also want to record the state of the particles so we can have a look at the trajectories our particles have taken during the simulation and process our data.

The cell below shows another simulation class skeleton, which has added a record_state method. This method should loop over all the particles and append a copy of them position to the state list. Once done it should append the state list to the trajectory attribute. You'll want to call the record_state method at the end of step method so that every time we update the particles we record them. Update your Simulation class to include the record_state method and call it from your step method.

    class Simulation():
        def __init__(self, particles, box_length, dt):
            self.trajectory = []

        def step(self):
            pass

        def record_state(self):
            state = []

Test your code using the cell below:

In [4]:
particles = [Particle(Vector(10,0),Vector(0,0.75),1,1),
             Particle(Vector(50,10),Vector(0,0.5),1,1),
             Particle(Vector(0,0), Vector(0.5,0.75),1,1)]

no_steps = 300

s=Simulation(particles, 100, 0.5)
for i in range(no_steps):
    s.step()

animate_trajectory(s)

## Automatically constructing Particle objects

Rather than manually defining particles let's construct a helper function *get_particles* that will generate an initial list of particles for us. The function will take two arguments, the number of particles we are creating and the length of the (square) box they are going to be in. It should return a list of particles each with a random position within the box and a random momenta. 

You'll want the function to assign momenta with components varying between -0.5 and 0.5 and positions with components between varying between 0.1$*$boxlength and 0.9$*$ boxlength

To make your function you'll need a way to generate some random numbers, fortunately there is a function *random* that does just this within the numpy library. Fill out the get_particles function below and se this function to simulate 100 particles for 500 time steps with dt set to 0.1. Finally pass your simulation object to the animate_trajectory function to see your simulation.

In [2]:
import numpy as np 
np.random.normal()
np.random.random

In [14]:
from numpy.random import random

def get_particles(no_particles, box_length):
    pass

# Collisions

We're making great progress but our simulation faces two problems:

   * Our particles pass through the boundaries of our box
   * Our particles pass through each other.
   
Let's focus on the boundaries problem to begin with. First execute the simple test we have below involving particles going through the boundary. We'll use this to test our code is behaving the way we want.

## Boundary Collision

In [61]:
def test_box():
    box_length = 100
    no_steps = 300
    
    p1 = Particle(position=Vector(10,50), momentum=Vector(-1,0),radius=1,mass=1)
    p2 = Particle(position=Vector(50,90), momentum=Vector(0,1) ,radius=1,mass=1)
    p3 = Particle(position=Vector(50,10), momentum=Vector(0,-1),radius=1,mass=1)
    p4 = Particle(position=Vector(90,50), momentum=Vector(1,0) ,radius=1,mass=1)

    box_particles = [p1,p2,p3,p4]

    s = Simulation(box_particles, box_length=100, dt=0.1)
    for i in range(300):
        s.step()
    animate_trajectory(s,loop=True)

test_box()

Ok, let's update our simulation class so that instead of passing through our box they bounce off the edges. Our simulation class will now make use of the previously unused attribute *box_length* and it will need an additional method *apply_box_collisions*.

At the moment our particles' momenta remain unaltered no matter what happens. But we would like the momentum of a particle along the x-axis to reverse if it goes past the sides of our box, likewise we would like the momentum of a particle along the y-axis to reverse if it goes past the top or the bottom of our box.

Try implementing an apply_box_collisions method as outlined in the skeleton below. This method will recieve as it's argument a particle and then it will check whether that particle is past the boundaries of the box and if it is change it's momentum accordingly.

We'll then add in a call to apply_box_collisions in our step method that loops over all the particles.

    class Simulation():
        def __init__(self, particles, box_length, dt):        
            self.trajectory = []

            pass

        def apply_box_collisions(self, particle):
            pass

        def step(self):
            pass

        def record_state(self):
            positions = []

            pass

Test your simulation class by executing the cell below

In [62]:
test_box()

## Particle Collisions

Ok the final problem and the most challenging is implementing interparticle collisions. First the following test cases:

In [63]:
def test_1d_collision():
    box_length = 100
    no_steps = 300

    p1 = Particle(position = Vector(10,10), momentum=Vector(1,0),radius=1,mass=1)
    p2 = Particle(position = Vector(20,10), momentum=Vector(0,0) ,radius=1,mass=1)
    
    test_case = [p1,p2]
    s=Simulation(particles=test_case,box_length=100,dt=0.05)

    for i in range(500):
        s.step()
    animate_trajectory(s,loop=True)

test_1d_collision()

In [64]:
def test_2d_collision():
    box_length = 100
    no_steps = 300

    p1 = Particle(position = Vector(10,10), momentum=Vector(1,0.5),radius=1,mass=1)
    p2 = Particle(position = Vector(20,15), momentum=Vector(0,0) ,radius=1,mass=1)
    
    test_case = [p1,p2]
    s=Simulation(particles=test_case,box_length=100,dt=0.05)

    for i in range(500):
        s.step()
    animate_trajectory(s,loop=True)

test_2d_collision()

### Enhancing the Particle class

One thing that will make our life easier is if we add another method to our particle class. In order to check whether two particles are going to collide we need to check whether they overlap. So we'll want to add an overlap method to our Particle class, this method will recieve as it's second argument another particle (as always the first argument a method recieves is always self). Our method will then return True if the distance between this other particle and itself is less than the sum of the two particle's radii otherwise it will return False. Extend the class in gas.py in accordance with the class skeleton below:

    class Particle():
        def __init__(self, position, momentum, radius, mass):
            pass

        def velocity(self):
            pass
        
        def copy(self):
            pass
            
        def overlap(self, other_particle):
            pass

In the cell below create some particle objects and see if your overlap method behaves the way you expect it to.

### 1D collisions

Ok so now we can think about the collisions themselves. We're going to focus on perfectly elastic collisions and to start with we're going to make two additional simplifications: 

* We're going to pretend that we're dealing with 1 dimensional particles that only move along the x-axis.
* We're going to pretend that the masses of both particles are equal. 

Under these conditions the result of an elastic collision is that the momenta of the two particles simply swap over after the collision as illustrated from the following [wiki](https://en.wikipedia.org/wiki/Elastic_collision) animation:

<img src=https://upload.wikimedia.org/wikipedia/commons/c/c6/Elastischer_sto%C3%9F.gif><img>

I.e.: 
$$p_{1}: mv_{1} \rightarrow mv_2$$ 
$$p_{2}: mv_{2} \rightarrow mv_1$$ 

Our particles are 2d as they have both x and y components for position and momentum, but if we make sure all the particles we're dealing with have the same y position and have 0 y momenta then they are effectively behaving like 1d particles.

Add a method *apply_particle_collision* to your simulation class that recieves as its arguments two particles and if they overlap alters their momentum along the x-axis in line with an elastic collision.


    class Simulation():
        def __init__(self, particles, dt, box_length):
            self.trajectory = []

            pass

        def apply_particle_collision(self, particle1, particle2):
            pass

        def apply_box_collisions(self, particle):
            pass

        def step(self):
            pass

        def record_state(self):
            positions = []

            pass

In the cell below create a few particle objects and check that your method behaves the way you expect it to.

Now we want to add in applying collisions to our step method. But to do that we need to loop over all pairs of particles. Note that we want to avoid colliding particle i with particle j and then also seperately colliding particle j with particle i - if we collide the same particles twice we will end up swapping their momenta twice hence undoing the collision.

There are several different ways of looping over pairs, but perhaps the easiest is to use the function **combinations** from the itertools library. See example below:

In [ ]:
from itertools import combinations

example_list = [1,2,3,4,5]
for p1,p2 in combinations(example_list,2):
    print(p1,p2)

Amend your step method so that it calls apply_particle_collision on all pairs of particles (you will need to import combinations at the top of your gas.py file)  and test your solution below:

In [43]:
test_1d_collision()

Since we've started adding more to our step method now is a good time to tidy it up a bit by creating an *update_position* method, moving the code that updates the positions from the step to update_postion and then calling that update_posistion from inside the step method.

    class Simulation():
        def __init__(self, particles, dt, box_length):
            self.trajectory = []

            pass

        def apply_particle_collision(self, particle1, particle2):
            pass

        def apply_box_collisions(self, particle):
            pass

        def update_positions(self, particle):
            pass
            
        def step(self):
            pass

        def record_state(self):
            positions = []

            pass
            
This process of reorganising code to make it clearer and easier to modify is known as **refactoring** and is an extremely important part of programming that is often overlooked by beginner programmers. We came across it before when we changed the Molecule class from being componsed of lists of different symbols, vectors and properties to being composed of a lists of Atom objects.

Make sure your refactor didn't break your code by re-executing the test_case below:

In [ ]:
test_1d_collision()

### 2D Collisions

Let's move on to 2d collisions! Currently we represent our momentumn in terms of two directions x and y. What we're going to do for our 2d case is to work out a different representation of the momentum in terms of two different directions. We will choose these directions in such a way that the collision will leave the second component completely unchanged, this essentially turns the 2d problem into the 1d problem we already know how to solve.

What are these two directions? The first is the direction of the displacement vector between the two particles at the moment they are colliding. We'll call this the *normal axis*,the other will be at 90 degrees to the normal and we'll call the *tangent axis*.

What we expect in our simplified case then is that following collision the normal components of the momentum of the particles will swap over whilst the tangent components will remain unchanged.

To make this a bit clearer let's look at the following animation from [wiki](https://en.wikipedia.org/wiki/Elastic_collision):

<img src=https://upload.wikimedia.org/wikipedia/commons/2/2c/Elastischer_sto%C3%9F_2D.gif></img>

What you are seeing is one particle in motion and another at rest. At the point where the two particles collide, the animation displays the normal and tangent axes as perpendicular black lines. 

The momentum of the moving particle (displayed as a red arrow) is expressed as components along these two axes (these are displayed as two blue arrows).

The stationary particle initally has no momentum which means, which means that both its normal component and its tangent component are initally zero.

After the collision the two particles swap normal components.

For the initially moving particle, the normal component becomes zero, leaving only its initial tangent component contributing to its final momentum. 

For the initially stationary particle, the initial tangent component is zero so the final momentum is equal to the normal component of initially moving particle prior to collision.

You may need to read this section a few times and stare at the animation for it to make sense. If after doing so it is still unclear please ask a demonstrator!

### Projecting the momentum along the normal and tangent axes

Luckily our vector class includes all the necessary methods to calculate the normal and tangent axes and express the momentum in terms of components along them.

To workout the component of a vector in the direction of a particular axis, we need only calculate the dot product of the vector with that particular axis multipled by the axis vector itself. Shown below for a 3 dimensional vector **a**, axis vectors **i**,**j**, and **k** and vector components $\textbf a_x$, $\textbf a_y$ and $\textbf a_z$.

<img src=https://upload.wikimedia.org/wikipedia/commons/thumb/f/fd/3D_Vector.svg/450px-3D_Vector.svg.png></img>

E.g. to calculate the component of a particular vector *vec* along the x-axis we would compute:
    
    vec = Vector(3,4)

    xaxis = Vector(1,0)
    vec_proj_x = xaxis * vec.dot(xaxis)
    print(vec_proj_x)

Like wise to calculate the component of a vector *vec* along the y-axis we would compute:
    
    yaxis = Vector(0,1)
    vec_proj_y = yaxis * vec.dot(yaxis)
    print(vec_proj_y)

Of course for x and y we have an easier option for working out the magnitude of along the x and y axes: we can simply use the .x and .y attributes. However this only works for x and y, whereas our general dot product approach works for any axes with might like to consider.

    new_axis = Vector(0.6,0.8)
    vec_proj_new_axis = new_axis * vec.dot(new_axis)
    print(vec_proj_new_axis)

An important point to bear in mind is that an axis vector **must always have a length of 1**. If we have a vector whose direction we would like to use as an axis, we must first shrink it's length to 1 whilst keeping its direction the same (known as normalisation). To do this we can divide the vector by its own length:

    normalised_vec = vec/vec.norm()
    print(vec)
    print(normalised_vec)

Having chosen one axis we can work out the component in the direction of the axis orthogonal (at 90 degrees) to our chosen one by subtracting the component along our chosen axis from our vector.

    new_xaxis = Vector(0.6,0.8)
    vec_proj_newx = new_xaxis * vec.dot(new_xaxis)
    vec_proj_newy = vec - vec_proj_newx

We now have assembled all the pieces we need to update our apply_particle_collision method for 2d collisions.

For colliding particles you will want to: 

* compute the normal axis
* compute the projection of the momentum onto the normal and tangent axes
* update the normal projected momentum in light of the collision
* compute  the new total momentum

Have a go updating your simulation class then execute the cell below to test your code.

In [ ]:
test_2d_collision()

Now that we've tested our code on simple cases it's time to run it on a larger number of particles.

Generate the initial conditions for 200 particles within a 100 length box, with random positions and 0.5 momentum for both their x and y components for all particles.

Create a simulation in a box of length 100 with dt 0.25 and execute 10,000 steps (it should take a few minutes or so to complete).

Animate your trajectory using the cell below

## Fixing a bug

Ok looking good but you might see a problem where some particles seem to be stuck together. Think about what would happen if you initialised your particles so that they started off moving slowly but already overlaping with one another? Do you see why you they would seem to stick together? There are two approaches to solving this problem:

Make sure when you construct your intitial particles they don't overlap, or when colliding make sure that the particles are actually travelling before swapping their momenta. To do the later you will need to project the difference in the particle momenta onto the normal axis and then check the sign of this projected difference in momenta.

Implement one of these solutions and check it works as intended. Finally recreate a simulation of 200 particles in 100 length box with 0.5 for the x/y components of the momenta then simulate 10,000 steps with dt 0.25 

Animate your trajectory using the cell below

**Congratulations you've written a particle simulation!**

The remaining part of the project is unguided so you're on your own from here!

## Extras

Using your simulation trajectory to produce a plot of total energy versus time for the simulation.

Every recorded state defines a distribution of particle speeds. 

Below is a skeleton for a function *plot_speeds* that accepts as its argument an integer defining the particular time step we are interested in and will plot the histogram of speeds present at that time step. (You may need to run %matplotlib to get the interactive plot to work) Fill out this function and use it to find out how many time steps our simulation takes till the distribution stops changing.

In [11]:
from ipywidgets import interact
import matplotlib.pyplot as plt

@interact(t=(0,1000,1))
def plot_speeds(t):
    pass

To get a better distribution create a simulation with a thousand particles and run it for a few hundred time steps (it might be a little slow, so you might need to reduce the time steps to as little as 500 - you can also try importing the Particle class from oo_functions as this is a faster version of the Particle class that you're currently using.)

The analytical formula for the 2D Maxwell-Boltzmann distribution giving the probabilty of finding a particle with a particular speed $v$ is:
    
$$p(v)=\frac{m v}{k_B T} \exp\left(-\frac{mv^2}{2k_BT}\right)$$ 

Where $m$ is the mass, $k_B$ is Boltzmann's constant and $T$ is the temperature.

The relationship between the temperature and the average energy for a 2d ideal gas is:

$$E_{avg} = k_BT$$

Workout the average energy per particle for your simulation above and use this value along with the mass you gave your particles to produce a plot of p(v) against v. (You should replace the p_v variable below with the values of p(v) predicted using the above equation).

By comparing the plot you produce with the histogram of speeds extracted from your 1000 particle simulation above do you think our numerical simulation agrees or disagrees with the analytical expression?

In [13]:
v = np.linspace(0,2,100)
p_v = 0

plt.plot(v,p_v)
plt.show()

We have assumed that all particles have the same mass and hence that:

$$p'_{1} = p_2 $$
$$p'_{2} = p_1 $$

Where the subscripts indicate the particle. p' is the momentum after the collision and p is the momentum before the collision.

But there is no reason to make this assumption. The correct formula for a 1D collision in the case of different particle masses is:
    
$$p'_{1} = \frac{p_{1}(m_{1}-m_{2})+2m_{1}p_{2}}{m_{1}+m_{2}}$$
$$p'_{2} = \frac{p_{2}(m_{2}-m_{1})+2m_{2}p_{1}}{m_{1}+m_{2}}$$

Update your simulation so that it can handle cases where particles have different masses and test it out on a suitable test case of your own devising.

## Optional (hard!)

**mixing**

Can you modify the boundary code so that you can record the pressure being exerted on the box during each time step and hence produce a plot of pressure vs. time step? Does your simulation agree with the ideal gas equations?

Can you modify the boundary code so that rather than perfect elastic collisions the boundary adds a random kick to the particles consistent with boundary being at some particular temperature $T_{boundary}$ (you will need to a Temperature attribute to the simulation class)

Our simulation is quite slow (you can see where the code is spending its time by including %%prun at the top of a cell and then executing your simulation code). Most of the computational expense is in the particle collisions because there are $\frac{N(N-1)}{2}$ pairs of particles. One way to reduce this expense is to divide the box up into quadrants, keep track of which quadrant the particles are in and then when executing the particle_collision code only look for collisions between pairs of particles in the same quadrant. Care needs to be taken when thinking about particles on the edges of the quadrants. Can you speed up the code in this way?

In [ ]:
%%prun

pass